# Task - 2  Forecasting sales of Superstore


# **Importing libraries**

In [4]:
import numpy as np
import pandas as pd

In [5]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot,init_notebook_mode,plot

init_notebook_mode(connected=True)

%matplotlib inline

# **Importing Data**

In [6]:
cd "C:\Users\Administrator\Desktop\internship"

C:\Users\Administrator\Desktop\internship


In [7]:
sales_data = pd.read_csv('train.csv')

In [8]:
sales_data.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [9]:
#Checking for Null values
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_tab = pd.concat([mis_val, mis_val_percent], axis=1)
        ren_mis_val_table_columns = mis_val_tab.rename(columns = {0 : 'Missing Values', 1 : '% of Total Missing Values'})
        ren_mis_val_table_columns = ren_mis_val_table_columns[ren_mis_val_table_columns.iloc[:,1] != 0].sort_values(
        '% of Total Missing Values', ascending=False).round(1)
        
        print ("\033[1mThe dataframe has total " + str(df.shape[1]) + " columns.\033[0m \n")
        print ("\033[1mThere are " 
               + str(ren_mis_val_table_columns.shape[0]) + " columns that have missing values.\033[0m \n")
        
        return ren_mis_val_table_columns

In [10]:
missing_val_sales = missing_values_table(sales_data)
missing_val_sales.style.background_gradient(cmap='Oranges')

The dataframe has total 18 columns. 

There are 1 columns that have missing values. 



,Missing Values,% of Total Missing Values
Postal Code,11,0.100000


# **Data Cleaning and Structuring**

In [11]:
"""
We need to remove the columns that is not required for our analysis
We need to check the null values. Fill them if possible or remove them
Also we need to check data type of each columns and change them according to our requirement for analysis
"""
sales_data.drop(['Row ID','Customer ID','Country','Product ID'],axis = 1, inplace = True)
sales_data['Order Date'] = pd.to_datetime(sales_data['Order Date'], format='%d/%m/%Y')
sales_data['Ship Date'] = pd.to_datetime(sales_data['Ship Date'], format='%d/%m/%Y') 
sales_data.set_index('Order ID',inplace=True)

In [12]:
#null values
sales_data[sales_data['Postal Code'].isnull()]

,Order Date,Ship Date,Ship Mode,Customer Name,Segment,City,State,Postal Code,Region,Category,Sub-Category,Product Name,Sales
Order ID,,,,,,,,,,,,,
CA-2018-104066,2018-12-05,2018-12-10,Standard Class,Quincy Jones,Corporate,Burlington,Vermont,NaN,East,Technology,Accessories,Logitech ClearChat Comfort/USB Headset H390,205.03
CA-2016-162887,2016-11-07,2016-11-09,Second Class,Stewart Visinsky,Consumer,Burlington,Vermont,NaN,East,Furniture,Chairs,Safco Contoured Stacking Chairs,715.20
US-2017-150140,2017-04-06,2017-04-10,Standard Class,Valerie Mitchum,Home Office,Burlington,Vermont,NaN,East,Technology,Phones,Nortel Meridian M5316 Digital phone,1294.75
US-2017-165505,2017-01-23,2017-01-27,Standard Class,Claudia Bergmann,Corporate,Burlington,Vermont,NaN,East,Technology,Accessories,Logitech Wireless Marathon Mouse M705,99.98
US-2017-165505,2017-01-23,2017-01-27,Standard Class,Claudia Bergmann,Corporate,Burlington,Vermont,NaN,East,Office Supplies,Art,4009 Highlighters,8.04
US-2017-165505,2017-01-23,2017-01-27,Standard Class,Claudia Bergmann,Corporate,Burlington,Vermont,NaN,East,Office Supplies,Storage,Iceberg Mobile Mega Data/Printer Cart,1564.29
US-2018-127292,2018-01-19,2018-01-23,Standard Class,Raymond Messe,Consumer,Burlington,Vermont,NaN,East,Office Supplies,Paper,Xerox 191,79.92
US-2018-127292,2018-01-19,2018-01-23,Standard Class,Raymond Messe,Consumer,Burlington,Vermont,NaN,East,Office Supplies,Paper,Xerox 1881,12.28
US-2018-127292,2018-01-19,2018-01-23,Standard Class,Raymond Messe,Consumer,Burlington,Vermont,NaN,East,Office Supplies,Appliances,Avanti 4.4 Cu. Ft. Refrigerator,542.94


In [13]:
# Postal code for Burlington city is 5401
sales_data['Postal Code'] = sales_data['Postal Code'].fillna(5401) 

In [14]:
sales_data.head(3)

,Order Date,Ship Date,Ship Mode,Customer Name,Segment,City,State,Postal Code,Region,Category,Sub-Category,Product Name,Sales
Order ID,,,,,,,,,,,,,
CA-2017-152156,2017-11-08,2017-11-11,Second Class,Claire Gute,Consumer,Henderson,Kentucky,42420.0,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
CA-2017-152156,2017-11-08,2017-11-11,Second Class,Claire Gute,Consumer,Henderson,Kentucky,42420.0,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
CA-2017-138688,2017-06-12,2017-06-16,Second Class,Darrin Van Huff,Corporate,Los Angeles,California,90036.0,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62


In [15]:
print('\033[1mSales Data Info :\033[0m \n')
print(sales_data.info())

Sales Data Info : 

<class 'pandas.core.frame.DataFrame'>
Index: 9800 entries, CA-2017-152156 to CA-2016-128608
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order Date     9800 non-null   datetime64[ns]
 1   Ship Date      9800 non-null   datetime64[ns]
 2   Ship Mode      9800 non-null   object        
 3   Customer Name  9800 non-null   object        
 4   Segment        9800 non-null   object        
 5   City           9800 non-null   object        
 6   State          9800 non-null   object        
 7   Postal Code    9800 non-null   float64       
 8   Region         9800 non-null   object        
 9   Category       9800 non-null   object        
 10  Sub-Category   9800 non-null   object        
 11  Product Name   9800 non-null   object        
 12  Sales          9800 non-null   float64       
dtypes: datetime64[ns](2), float64(2), object(9)
memory usage: 1.0+ MB
None


In [16]:
print('\033[1mSales Data Shape(Columns , Rows) :\033[0m \n')
print(sales_data.shape)

Sales Data Shape(Columns , Rows) : 

(9800, 13)


In [17]:
print('\033[1mSales Data Statistic :\033[0m \n')
print(sales_data['Sales'].describe())

Sales Data Statistic : 

count     9800.000000
mean       230.769059
std        626.651875
min          0.444000
25%         17.248000
50%         54.490000
75%        210.605000
max      22638.480000
Name: Sales, dtype: float64


In [18]:
print('\033[1mFirst Sales Order date :\033[0m \n')
print(f"\033[1m{sales_data['Order Date'].min()}\033[0m")

First Sales Order date : 

2015-01-03 00:00:00


In [19]:
print('\033[1mLast Sales Order date :\033[0m \n')
print(f"\033[1m{sales_data['Order Date'].max()}\033[0m")

Last Sales Order date : 

2018-12-30 00:00:00


# **Sales Date Analysis**

In [21]:
sales_data['Month_Year'] = sales_data['Order Date'].apply(lambda x: x.strftime('%Y-%m'))
sales_data['Year'] = sales_data['Order Date'].apply(lambda x: x.strftime('%Y'))

In [43]:
monthly_trend = sales_data.groupby(['Month_Year']).sum().drop('Postal Code', axis=1).round(2)
monthly_trend.reset_index(inplace=True)

# Change the color of the markers
colr = np.arange(0, monthly_trend.values.size)
marker_color = []
for i in range(len(colr)):
    if i % 2 == 0:
        marker_color.append('#0000FF')
    else:
        marker_color.append('#FF0000')

fig = go.Figure(data=[go.Scatter(x=monthly_trend['Month_Year'], y=monthly_trend['Sales'], mode='lines',
                                 marker=dict(color=marker_color))], layout=go.Layout(title='Total Monthly Sales'))
fig.update_traces(hoverinfo='x+y', textfont_size=10)
fig.update_layout(height=600, autosize=True, plot_bgcolor='rgb(205, 275, 275)')

> **Total Yealy Sales**

In [44]:
yearly_trend = sales_data.groupby(['Year']).sum().drop('Postal Code',axis=1).round(2)

print('\033[1mTotal yearly sales :\033[0m \n')
print(f'\033[1m{yearly_trend}\033[0m \n')
yearly_trend.reset_index(inplace=True)

colr=np.arange(0,yearly_trend.values.size)
fig=go.Figure(data=[go.Scatter(x=yearly_trend['Year'] ,y=yearly_trend['Sales'],mode='lines', marker=dict(
                color=colr))],layout=go.Layout(title='Total Yealy Sales'))
fig.update_traces(hoverinfo='x+y', textfont_size=10)               
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 275, 275)')

Total yearly sales : 

          Sales
Year           
2015  479856.21
2016  459436.01
2017  600192.55
2018  722052.02 



> **Average Monthly Sales**

In [45]:
avg_monthly_trend = sales_data.groupby(['Month_Year']).mean().drop('Postal Code',axis=1).round(2)

print('\033[1mAverage monthly sales :\033[0m \n')
print(f'\033[1m{avg_monthly_trend}\033[0m \n')
avg_monthly_trend.reset_index(inplace=True)

colr=np.arange(0,avg_monthly_trend.values.size)
fig=go.Figure(data=[go.Scatter(x=avg_monthly_trend['Month_Year'] ,y=avg_monthly_trend['Sales'],mode='lines', marker=dict(
                color=colr))],layout=go.Layout(title='Average Monthly Sales'))
fig.update_traces(hoverinfo='x+y', textfont_size=10)               
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 275, 275)')

Average monthly sales : 

             Sales
Month_Year        
2015-01     184.49
2015-02      98.26
2015-03     358.48
2015-04     214.67
2015-05     195.41
2015-06     262.01
2015-07     237.90
2015-08     185.74
2015-09     305.71
2015-10     197.82
2015-11     254.60
2015-12     248.78
2016-01     316.96
2016-02     186.74
2016-03     252.65
2016-04     214.81
2016-05     209.51
2016-06     182.94
2016-07     210.35
2016-08     233.03
2016-09     221.52
2016-10     189.10
2016-11     234.42
2016-12     239.69
2017-01     208.34
2017-02     276.85
2017-03     317.80
2017-04     230.24
2017-05     256.37
2017-06     202.68
2017-07     195.51
2017-08     174.53
2017-09     198.26
2017-10     310.33
2017-11     216.03
2017-12     283.25
2018-01     304.03
2018-02     191.55
2018-03     248.37
2018-04     177.71
2018-05     182.61
2018-06     205.07
2018-07     201.01
2018-08     293.63
2018-09     190.18
2018-10     263.43
2018-11     258.64
2018-12     180.89 



> **Average Yealy Sales**

In [46]:
avg_yearly_trend = sales_data.groupby(['Year']).mean().drop('Postal Code',axis=1).round(2)

print('\033[1mAverage yearly sales :\033[0m \n')
print(f'\033[1m{avg_yearly_trend}\033[0m \n')
avg_yearly_trend.reset_index(inplace=True)

colr=np.arange(0,avg_yearly_trend.values.size)
fig=go.Figure(data=[go.Scatter(x=avg_yearly_trend['Year'] ,y=avg_yearly_trend['Sales'],mode='lines', marker=dict(
                color=colr))],layout=go.Layout(title='Average Yealy Sales'))
fig.update_traces(hoverinfo='x+y', textfont_size=10)               
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 275, 275)')

Average yearly sales : 

       Sales
Year        
2015  245.70
2016  223.57
2017  236.86
2018  221.62 



# **Product Analysis**

In [47]:
top_products = sales_data.groupby(['Product Name']).sum().sort_values(by="Sales", ascending=False).head(20).drop('Postal Code',axis=1).round(2)

print('\033[1mTop 20 Products by sales :\033[0m \n')
print(f'\033[1m{top_products}\033[0m \n')

top_products.reset_index(inplace=True)
colr=np.arange(0,top_products.size)
fig=go.Figure(data=[go.Bar(y=top_products['Sales'],x=top_products['Product Name'], marker={'color':colr})], layout=go.Layout(title='Most Valuable Products by Sales',
          xaxis=dict(title='Product Name'),
          yaxis = dict(title='Sales made')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 275, 275)')

Top 20 Products by sales : 

                                                       Sales
Product Name                                                
Canon imageCLASS 2200 Advanced Copier               61599.82
Fellowes PB500 Electric Punch Plastic Comb Bind...  27453.38
Cisco TelePresence System EX90 Videoconferencin...  22638.48
HON 5400 Series Task Chairs for Big and Tall        21870.58
GBC DocuBind TL300 Electric Binding System          19823.48
GBC Ibimaster 500 Manual ProClick Binding System    19024.50
Hewlett Packard LaserJet 3310 Copier                18839.69
HP Designjet T520 Inkjet Large Format Printer -...  18374.90
GBC DocuBind P400 Electric Binding System           17965.07
High Speed Automatic Electric Letter Opener         17030.31
Lexmark MX611dhe Monochrome Laser Printer           16829.90
Martin Yale Chadless Opener Electric Letter Opener  16656.20
Ibico EPK-21 Electric Binding System                15875.92
Riverside Palais Royal Lawyers Bookcase, Royale...  1561

In [49]:
most_sold_products = sales_data['Product Name'].value_counts().head(20)

print('\033[1mTop 20 Products by quantity :\033[0m \n')
print(f'\033[1m{most_sold_products}\033[0m \n')

# most_sold_products.reset_index(inplace=True)
colr=np.arange(0,most_sold_products.size)
fig=go.Figure(data=[go.Bar(y=most_sold_products.values,x=most_sold_products.index, marker={'color':colr})], layout=go.Layout(title='Most selling Products',
          xaxis=dict(title='Product Name'),
          yaxis = dict(title='Number items sold')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

Top 20 Products by quantity : 

Staple envelope                                               47
Staples                                                       46
Easy-staple paper                                             44
Avery Non-Stick Binders                                       20
Staples in misc. colors                                       18
Staple remover                                                18
Storex Dura Pro Binders                                       17
KI Adjustable-Height Table                                    17
Staple-based wall hangings                                    16
Logitech 910-002974 M325 Wireless Mouse for Web Scrolling     15
Situations Contoured Folding Chairs, 4/Set                    15
Eldon Wave Desk Accessories                                   14
Global Wood Trimmed Manager's Task Chair, Khaki               14
GBC Premium Transparent Covers with Diagonal Lined Pattern    14
Global High-Back Leather Tilter, Burgundy                 

In [50]:
least_products = sales_data.groupby(['Product Name']).sum().sort_values(by="Sales", ascending=False).tail(20).drop('Postal Code',axis=1).round(2)

print('\033[1mLeast 20 Products by sales :\033[0m \n')
print(f'\033[1m{least_products}\033[0m \n')

least_products.reset_index(inplace=True)
colr=np.arange(0,least_products.size)
fig=go.Figure(data=[go.Bar(y=least_products['Sales'],x=least_products['Product Name'], marker={'color':colr})], layout=go.Layout(title='Least Valuable Products by Sales',
          xaxis=dict(title='Product Name'),
          yaxis = dict(title='Sales made')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

Least 20 Products by sales : 

                                                    Sales
Product Name                                             
Newell 337                                          10.50
Letter Slitter                                      10.08
Alliance Big Bands Rubber Bands, 12/Pack             9.90
Newell 342                                           9.84
Hoover Replacement Belt for Commercial Guardsma...   9.77
PNY Rapid USB Car Charger - Black                    9.59
Kleencut Forged Office Shears by Acme United Co...   9.57
Avery 484                                            9.22
Newell 308                                           8.40
Acco Economy Flexible Poly Round Ring Binder         8.35
Stockwell Gold Paper Clips                           8.10
4009 Highlighters                                    8.04
Xerox 1989                                           7.97
Avery Hi-Liter Comfort Grip Fluorescent Highlig...   7.80
Avery Hi-Liter Pen Style Six-Color Fluore

In [53]:
least_sold_products = sales_data['Product Name'].value_counts().tail(20)

print('\033[1mLeast 20 Products by quantity :\033[0m \n')
print(f'\033[1m{least_sold_products}\033[0m \n')

colr=np.arange(0,least_sold_products.size)
fig=go.Figure(data=[go.Bar(y=least_sold_products.values,x=least_sold_products.index, marker={'color':colr})], layout=go.Layout(title='Least selling Products',
          xaxis=dict(title='Product Name'),
          yaxis = dict(title='Number items sold')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

Least 20 Products by quantity : 

Xerox 215                                                              1
RCA ViSYS 25423RE1 Corded phone                                        1
Avaya IP Phone 1140E VoIP phone                                        1
Linden 12" Wall Clock With Oak Frame                                   1
Konica Minolta magicolor 1690MF Multifunction Printer                  1
Xerox 1922                                                             1
AT&T EL51110 DECT                                                      1
Hewlett-Packard Deskjet 3050a All-in-One Color Inkjet Printer          1
Hunt BOSTON Model 1606 High-Volume Electric Pencil Sharpener, Beige    1
SanDisk Cruzer 8 GB USB Flash Drive                                    1
Ultra Commercial Grade Dual Valve Door Closer                          1
Cisco SPA525G2 5-Line IP Phone                                         1
Wasp CCD Handheld Bar Code Reader                                      1
Computer Printout

# **Customers Analysis**

In [56]:
top_customers = sales_data.groupby(['Customer Name']).sum().sort_values(by='Sales', ascending=False).head(20).drop('Postal Code',axis=1).round(2)

print('\033[1mTop 20 Customers :\033[0m \n')
print(f'\033[1m{top_customers}\033[0m \n')

top_customers.reset_index(inplace=True)
colr=np.arange(0,top_customers.size)
fig=go.Figure(data=[go.Bar(y=top_customers['Sales'],x=top_customers['Customer Name'], marker={'color':colr})], layout=go.Layout(title='Most Valuable Customers',
          xaxis=dict(title='Customers'),
          yaxis = dict(title='Sales made')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

Top 20 Customers : 

                       Sales
Customer Name               
Sean Miller         25043.05
Tamara Chand        19052.22
Raymond Buch        15117.34
Tom Ashbrook        14595.62
Adrian Barton       14473.57
Ken Lonsdale        14175.23
Sanjit Chand        14142.33
Hunter Lopez        12873.30
Sanjit Engle        12209.44
Christopher Conant  12129.07
Todd Sumrall        11891.75
Greg Tran           11820.12
Becky Martin        11789.63
Seth Vernon         11470.95
Caroline Jumper     11164.97
Clay Ludtke         10880.55
Maria Etezadi       10663.73
Karen Ferguson      10604.27
Bill Shonely        10501.65
Edward Hooks         9940.38 



In [57]:
least_customers = sales_data.groupby(['Customer Name']).sum().sort_values(by='Sales', ascending=False).tail(20).drop('Postal Code',axis=1).round(2)

print('\033[1m20 Customers with Least Revenue :\033[0m \n')
print(f'\033[1m{least_customers}\033[0m \n')

least_customers.reset_index(inplace=True)
colr=np.arange(0,least_customers.size)
fig=go.Figure(data=[go.Bar(y=least_customers['Sales'],x=least_customers['Customer Name'], marker={'color':colr})], layout=go.Layout(title='Least Revenue generated by Customers',
          xaxis=dict(title='Customers'),
          yaxis = dict(title='Sales made')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

20 Customers with Least Revenue : 

                  Sales
Customer Name          
Phillip Breyer   132.74
Bobby Odegard    130.83
Ed Ludwig        124.28
Clay Cheatham    113.83
Roland Murray     98.35
Karen Seio        88.47
Anemone Ratner    88.15
Fred Wasserman    79.75
Jasper Cacioppo   71.26
Adrian Shami      58.82
Larry Blacks      50.19
Ionia McGrath     48.40
Ricardo Emerson   48.36
Susan Gilcrest    47.95
Roy Skaria        22.33
Mitch Gastineau   16.74
Carl Jackson      16.52
Sung Chung        14.11
Lela Donovan       5.30
Thais Sissman      4.83 



# **Shipping Mode Analysis**

In [58]:
ship_modes = sales_data['Ship Mode'].value_counts()

print('\033[1mMost used Shipping Modes :\033[0m \n')
print(f'\033[1m{ship_modes}\033[0m \n')

colr=np.arange(0,ship_modes.size)

fig=go.Figure(data=[go.Pie(labels=ship_modes.index,values=ship_modes.values)],layout=go.Layout(title='Most used Shipping Modes'))
fig.update_traces(hoverinfo='label+percent+value', textinfo='label+percent', textfont_size=10,marker=dict(colors=colr))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black"                
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

Most used Shipping Modes : 

Standard Class    5859
Second Class      1902
First Class       1501
Same Day           538
Name: Ship Mode, dtype: int64 



In [34]:
ship_modes_sales = sales_data.groupby(['Ship Mode']).sum().drop('Postal Code',axis=1).round(2)

print('\033[1mRevenue generated by Shipping Modes :\033[0m \n')
print(f'\033[1m{ship_modes_sales}\033[0m \n')
ship_modes_sales.reset_index(inplace=True)

colr=np.arange(0,ship_modes_sales.values.size)
fig=go.Figure(data=[go.Pie(labels=ship_modes_sales['Ship Mode'],values=ship_modes_sales['Sales'])],layout=go.Layout(title='Revenue generated by Shipping Modes'))
fig.update_traces(hoverinfo='label+percent', textinfo='label+value', textfont_size=10,marker=dict(colors=colr))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black"                
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(275, 275, 275)')

Revenue generated by Shipping Modes : 

                     Sales
Ship Mode                 
First Class      345572.26
Same Day         125219.04
Second Class     449914.18
Standard Class  1340831.31 



# **Sales Category and Sub-Category Analysis**

In [59]:
catagory = sales_data.groupby(['Category','Sub-Category']).sum().drop('Postal Code',axis=1).round(2)

print('\033[1mSales made by Category and Sub-Category :\033[0m \n')
print(f'\033[1m{catagory}\033[0m \n')

sub_cat1 = catagory.loc['Furniture'].reset_index()
sub_cat2 = catagory.loc['Office Supplies'].reset_index()
sub_cat3 = catagory.loc['Technology'].reset_index()

cat = sales_data['Category'].unique()

fig = make_subplots(rows=2, cols=2)

fig.add_trace(
    go.Bar(x=sub_cat1['Sub-Category'], y=sub_cat1['Sales'],marker={'color':np.arange(0,sub_cat1['Sub-Category'].size)},name=cat[0]),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=sub_cat2['Sub-Category'], y=sub_cat2['Sales'],marker={'color':np.arange(0,sub_cat2['Sub-Category'].size)},name=cat[1]),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=sub_cat3['Sub-Category'], y=sub_cat2['Sales'],marker={'color':np.arange(0,sub_cat3['Sub-Category'].size)},name=cat[2]),
    row=2, col=1
)

fig.update_xaxes(title_text=f"Category : {cat[0]}", row=1, col=1)
fig.update_xaxes(title_text=f"Category : {cat[1]}", row=1, col=2)
fig.update_xaxes(title_text=f"Category : {cat[2]}", row=2, col=1)

fig.update_yaxes(title_text="Sales Amount", row=1, col=1)
fig.update_yaxes(title_text="Sales Amount", row=1, col=2)
fig.update_yaxes(title_text="Sales Amount", row=2, col=1)

fig.update_layout(height=700, width=900, title_text="Sales Analysis of different Categories")

Sales made by Category and Sub-Category : 

                                  Sales
Category        Sub-Category           
Furniture       Bookcases     113813.20
                Chairs        322822.73
                Furnishings    89212.02
                Tables        202810.63
Office Supplies Appliances    104618.40
                Art            26705.41
                Binders       200028.78
                Envelopes      16128.05
                Fasteners       3001.96
                Labels         12347.73
                Paper          76828.30
                Storage       219343.39
                Supplies       46420.31
Technology      Accessories   164186.70
                Copiers       146248.09
                Machines      189238.63
                Phones        327782.45 



# **Geoanalytics**

In [60]:
top_states = sales_data.groupby(['State']).sum().sort_values(by='Sales', ascending=False).head(20).drop('Postal Code',axis=1).round(2)

print('\033[1mTop 20 States with Highest revenue :\033[0m \n')
print(f'\033[1m{top_states}\033[0m \n')
top_states.reset_index(inplace=True)

colr=np.arange(0,top_states.values.size)
fig=go.Figure(data=[go.Bar(y=top_states['Sales'],x=top_states['State'], marker={'color':colr})], layout=go.Layout(title='States with Highest Revenue',
          xaxis=dict(title='States'),
          yaxis = dict(title='Revenue')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

Top 20 States with Highest revenue : 

                    Sales
State                    
California      446306.46
New York        306361.15
Texas           168572.53
Washington      135206.85
Pennsylvania    116276.65
Florida          88436.53
Illinois         79236.52
Michigan         76136.07
Ohio             75130.35
Virginia         70636.72
North Carolina   55165.96
Indiana          48718.40
Georgia          48219.11
Kentucky         36458.39
Arizona          35272.66
New Jersey       34610.97
Colorado         31841.60
Wisconsin        31173.43
Tennessee        30661.87
Minnesota        29863.15 



In [37]:
top_cities = sales_data.groupby(['City']).sum().sort_values(by='Sales', ascending=False).head(20).drop('Postal Code',axis=1).round(2)

print('\033[1mTop 20 Cities with Highest revenue :\033[0m \n')
print(f'\033[1m{top_cities}\033[0m \n')
top_cities.reset_index(inplace=True)

colr=np.arange(0,top_cities.values.size)
fig=go.Figure(data=[go.Bar(y=top_cities['Sales'],x=top_cities['City'], marker={'color':colr})], layout=go.Layout(title='Cities with Highest Revenue',
          xaxis=dict(title='Cities'),
          yaxis = dict(title='Revenue')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(275, 275, 275)')

Top 20 Cities with Highest revenue : 

                   Sales
City                    
New York City  252462.55
Los Angeles    173420.18
Seattle        116106.32
San Francisco  109041.12
Philadelphia   108841.75
Houston         63956.14
Chicago         47820.13
San Diego       47521.03
Jacksonville    44713.18
Detroit         42446.94
Springfield     41827.81
Columbus        38662.56
Newark          28448.05
Columbia        25283.32
Jackson         24963.86
Lafayette       24944.28
San Antonio     21843.53
Burlington      21668.08
Arlington       20214.53
Dallas          20127.95 



> **Revenue generated by Region**

In [61]:
region_sales = sales_data.groupby(['Region']).sum().sort_values(by='Sales', ascending=False).drop('Postal Code',axis=1).round(2)

print('\033[1mRegion wise revenue :\033[0m \n')
print(f'\033[1m{region_sales}\033[0m \n')
region_sales.reset_index(inplace=True)

colr=np.arange(0,region_sales.values.size)
fig=go.Figure(data=[go.Pie(labels=region_sales['Region'],values=region_sales['Sales'])],layout=go.Layout(title='Revenue generated by Region'))
fig.update_traces(hoverinfo='label+percent+value', textinfo='label+percent', textfont_size=10,marker=dict(colors=colr))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black"                
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

Region wise revenue : 

             Sales
Region            
West     710219.68
East     669518.73
Central  492646.91
South    389151.46 



In [62]:
least_states = sales_data.groupby(['State']).sum().sort_values(by='Sales', ascending=False).tail(20).drop('Postal Code',axis=1).round(2)

print('\033[1m20 States with lowest revenue :\033[0m \n')
print(f'\033[1m{least_states}\033[0m \n')
least_states.reset_index(inplace=True)

colr=np.arange(0,least_states.values.size)
fig=go.Figure(data=[go.Bar(y=least_states['Sales'],x=least_states['State'], marker={'color':colr})], layout=go.Layout(title='States with lowest Revenue',
          xaxis=dict(title='States'),
          yaxis = dict(title='Revenue')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

20 States with lowest revenue : 

                         Sales
State                         
Connecticut           13384.36
Arkansas              11678.13
Utah                  11220.06
Mississippi           10771.34
Louisiana              9131.05
Vermont                8929.37
South Carolina         8481.71
Nebraska               7464.93
New Hampshire          7292.52
Montana                5589.35
New Mexico             4783.52
Iowa                   4443.56
Idaho                  4382.49
Kansas                 2914.31
District of Columbia   2865.02
Wyoming                1603.14
South Dakota           1315.56
Maine                  1270.53
West Virginia          1209.82
North Dakota            919.91 



> **Least Revenue generated by Cities**

In [63]:
least_cities = sales_data.groupby(['City']).sum().sort_values(by='Sales', ascending=False).tail(20).drop('Postal Code',axis=1).round(2)

print('\033[1m20 Cities with lowest revenue :\033[0m \n')
print(f'\033[1m{least_cities}\033[0m \n')
least_cities.reset_index(inplace=True)

colr=np.arange(0,least_cities.values.size)
fig=go.Figure(data=[go.Bar(y=least_cities['Sales'],x=least_cities['City'], marker={'color':colr})], layout=go.Layout(title='Cities with lowest Revenue',
          xaxis=dict(title='Cities'),
          yaxis = dict(title='Revenue')))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black" 
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

20 Cities with lowest revenue : 

                   Sales
City                    
New Brunswick      14.77
Arlington Heights  14.11
Chapel Hill        14.02
Rock Hill          11.85
Baytown            10.37
Iowa City           9.99
Romeoville          8.95
Billings            8.29
Port Orange         7.82
Deer Park           6.92
Missouri City       6.37
Keller              6.00
Layton              4.96
Springdale          4.30
San Luis Obispo     3.62
Ormond Beach        2.81
Pensacola           2.21
Jupiter             2.06
Elyria              1.82
Abilene             1.39 



# **Sales Segment Analysis**

In [67]:
segment_sales = sales_data.groupby(['Segment']).sum().sort_values(by='Sales', ascending=False).drop('Postal Code',axis=1).round(2)

print('\033[1mSegment wise revenue :\033[0m \n')
print(f'\033[1m{segment_sales}\033[0m \n')
segment_sales.reset_index(inplace=True)

colr=np.arange(0,segment_sales.values.size)
fig=go.Figure(data=[go.Pie(labels=segment_sales['Segment'],values=segment_sales['Sales'])],layout=go.Layout(title='Revenue generated by each Segment'))
fig.update_traces(hoverinfo='label+percent+value', textinfo='label+percent', textfont_size=10,marker=dict(colors=colr))
fig.data[0].marker.line.width = 2
fig.data[0].marker.line.color = "black"                
fig.update_layout(height=600,autosize=True ,plot_bgcolor='rgb(205, 205, 275)')

Segment wise revenue : 

                  Sales
Segment                
Consumer     1148060.53
Corporate     688494.07
Home Office   424982.18 

